<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/images/top.png?raw=true'>

# Configuration (à lancer avant tous les notebooks)

In [ ]:
# version de python
import platform
platform.python_version()

In [ ]:
# la liste des packages installés (on peut vérifier la présence des dépendances azure)
!conda list

In [ ]:
# version de la SDK azureml
import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

Si le notebook est executé en dehors d'Azure, il faut télécharger le fichier config.json depuis le portail https://portal.azure.com/, et le mettre dans le workspace qui contient le notebook.

Si le notebook est exécuté directement depuis le workspace Azure, le fichier de config devrait déjà être là.

In [ ]:
# connexion au workspace
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, "loaded")

# Envoyer les données sur la plateforme

In [ ]:
from azureml.core import Dataset

dataset_name = 'titanic train dataset'
description='titanic training data'
target_path='titanic-data/'

default_ds = ws.get_default_datastore()

if dataset_name not in ws.datasets:
    default_ds.upload_files(files=['data/titanic.csv'], target_path=target_path, overwrite=True, show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'titanic-data/titanic.csv'))

    # Register the tabular dataset
    try:
        tab_data_set.register(workspace=ws, 
                                name=dataset_name,
                                description=description,
                                tags = {'format':'CSV'},
                                create_new_version=True)
        
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

In [ ]:
titanic_ds = ws.datasets.get("titanic train dataset")
titanic_ds.to_pandas_dataframe().head()

# Executez le script dans une expérience

In [ ]:
from azureml.train.sklearn import SKLearn
from azureml.core import Experiment
from azureml.widgets import RunDetails

# Get the training dataset
titanic_ds = ws.datasets.get("titanic train dataset")

# Create an estimator
estimator = SKLearn(source_directory='script',
                    entry_script='titanic_training.py',
                    compute_target = 'local',
                    inputs=[titanic_ds.as_named_input('titanic')], # Pass the Dataset object as an input...
                    pip_packages=[] # extra needed packages
                   )

# Create an experiment
experiment_name = 'titanic-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# Run the experiment
run = experiment.submit(config=estimator)
# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()

# Ne pas oublier à la fin de l'expérience!!
(si votre travail à utilisé une instance de calcul)

<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/images/down.png?raw=true'>



## stoppe une machine à partir de son nom

In [ ]:
compute_name = "XXXX"

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]

    print('try to stop compute', compute.name)
        compute.stop(show_output=True)

else :
    print('compute target not found', compute.name)

## Liste tous les compute pour vérifier qu'elles sont éteintes

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance

# liste tous les compute pour vérifier qu'elles sont éteintes
for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance:
        print(compute.name, compute.get_status())


# Ressources

[doc SDK azure](https://docs.microsoft.com/en-us/azure/developer/python/azure-sdk-overview)

[doc SDK azureML](https://docs.microsoft.com/fr-fr/python/api/overview/azure/ml/install?view=azure-ml-py)

[api azure](https://docs.microsoft.com/en-us/python/api/azureml-core)

[parcours d'apprentissage microsoft](https://docs.microsoft.com/fr-fr/learn/paths/build-ai-solutions-with-azure-ml-service/)

[le repository microsoft](https://github.com/MicrosoftDocs/mslearn-aml-labs.git)